In [ ]:
from script import df_mobpro_brut, arr_marseille, arr_paris, arr_lyon, contours_comm, transport_dict, contours_comm, flux, plot_flux_gradient, plot_flux_gradient_zoom, coord_villes
from script import df_dossier_complet_brut
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import matplotlib.colors as colors
import networkx as nx
from scipy.sparse import coo_matrix

In [ ]:
df_mobpro_brut.head(3)

Ce que signifient chacune des en-têtes de colonnes :  
COMMUNE Département et commune du lieu de résidence  
ARM Arrondissement municipal de résidence (Paris, Lyon et Marseille)  
DCFLT Commune frontalière du lieu de travail  
DCLT Département, commune et arrondissement municipal (Paris, Lyon, Marseille) du lieu de travail  
AGEREVQ Âge quinquennal en années révolues  
CS1 Catégorie socioprofessionnelle en 8 postes  
DEROU Nombre de deux-roues à moteur du ménage (DOM)  
DIPL_15 Diplôme le plus élevé  
EMPL Condition d'emploi  
ILTUU Indicateur urbain du lieu de travail  
ILT Indicateur du lieu de travail  
IMMI Situation quant à l'immigration  
INATC Indicateur de nationalité condensé (Français/Étranger)  
INEEM Nombre d'élèves, étudiants ou stagiaires âgés de 14 ans ou plus du ménage  
INPOM Nombre de personnes actives ayant un emploi du ménage  
INPSM Nombre de personnes scolarisées du ménage  
IPONDI Poids de l'individu  
LPRM Lien à la personne de référence du ménage  
METRODOM Indicateur Métropole ou DOM du lieu de résidence  
MOCO Mode de cohabitation  
NA5 Activité économique regroupée en 5 postes  
NPERR Nombre de personnes du ménage (regroupé)  
REGION Région du lieu de résidence  
REGLT Région du lieu de travail  
SEXE Sexe  
STAT Statut professionnel  
STOCD Statut d'occupation détaillé du logement  
TP Temps de travail  
TRANS Mode de transport principal le plus souvent utilisé pour aller travailler  
TYPL Type de logement  
TYPMR Type de ménage regroupé (en 9 postes)  
VOIT Nombre de voitures du ménage  

In [ ]:
# Distribution des modes de transport utilisés pour aller travailler
transport_distribution = df_mobpro_brut['TRANS_LABEL'].value_counts()
print(transport_distribution)

# Visualisation
transport_distribution.plot(kind='bar', color='skyblue')
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.title('Distribution des modes de transport')
plt.xlabel('Mode de transport')
plt.ylabel('Nombre d’individus')
plt.show()

In [ ]:
# Âge moyen par mode de transport
age_transport = df_mobpro_brut.groupby('TRANS_LABEL')['AGEREVQ'].mean()
print(age_transport)

# Visualisation
age_transport.plot(kind='bar', color='orange')
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.title('Âge moyen par mode de transport')
plt.xlabel('Mode de transport')
plt.ylabel('Âge moyen')
plt.show()


In [ ]:
# Distribution des modes de transport par catégorie socio-professionnelle avec labels
cs1_transport = pd.crosstab(df_mobpro_brut['CS_LABEL'], df_mobpro_brut['TRANS_LABEL'])

# Heatmap
sns.heatmap(cs1_transport, annot=True, fmt='d', cmap='Blues')
plt.title('Modes de transport par catégorie socio-professionnelle')
plt.xlabel('Mode de transport')

# Inclinaison des labels avec alignement pour que l'extrémité pointe vers l'axe
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.ylabel('Catégorie socio-professionnelle')
plt.show()


La fonction ci-dessous permet d'obtenir un premier tableau regroupant certaines informations par commune :
- Nombre de personnes du ménage (regroupé)
- Nombre de personnes scolarisées du ménage
- Nombre de personnes actives ayant un emploi du ménage
- Nombre d'élèves, étudiants ou stagiaires âgés de 14 ans ou plus du ménage
- Indicateur du lieu de travail
(1 Dans la commune de résidence actuelle
2 Dans une autre commune du département de résidence
3 Dans un autre département de la région de résidence
4 Hors de la région de résidence actuelle : en métropole
5 Hors de la région de résidence actuelle : dans un DOM
6 Hors de la région de résidence actuelle : dans une COM
7 A l'étranger)

In [ ]:
# Conversion des colonnes en numérique
df_mobpro_brut['NPERR'] = pd.to_numeric(df_mobpro_brut['NPERR'], errors='coerce')
df_mobpro_brut['INPSM'] = pd.to_numeric(df_mobpro_brut['INPSM'], errors='coerce')
df_mobpro_brut['INPOM'] = pd.to_numeric(df_mobpro_brut['INPOM'], errors='coerce')
df_mobpro_brut['INEEM'] = pd.to_numeric(df_mobpro_brut['INEEM'], errors='coerce')

# Création des agrégations de base
mobpro_villes = df_mobpro_brut.groupby('COMMUNE').agg({
    'NPERR': 'sum',
    'INPSM': 'sum',
    'INPOM': 'sum',
    'INEEM': 'sum'})

# Créons d'abord un DataFrame temporaire pour chaque valeur d'ILT
for i in range(1, 8):
    # Grouper par COMMUNE et compter les occurrences où ILT == str(i)
    temp_count = df_mobpro_brut[df_mobpro_brut['ILT'] == i].groupby('COMMUNE').size()
    # Ajouter cette série au DataFrame principal
    mobpro_villes[f'ILT_{i}'] = temp_count

# Remplir les valeurs manquantes (NaN) par 0
mobpro_villes = mobpro_villes.fillna(0)

# Réinitialisation de l'index
mobpro_villes = mobpro_villes.reset_index()

In [ ]:
mobpro_villes.head()

On souhaite créer une matrice des flux pour avoir une vision générale des trajets domicile-travail quotidiens entre les villes. Pour chaque couple de ville, le nombre correspondant dans la matrice correspond au nombre de commute effectué de la ville 1 à la ville 2. 

On remarque que pour la variable "COMMUNE" (commune de résidence) on a uniquement 75056 pour Paris (idem pour Lyon et Marseille), tandis que pour la variable "DCLT" (commune de travail) on a un découpage en arrondissement avec les codes 75101,... pour Paris (idem pour Lyon et Marseille). Afin de pallier cette différence, on se cale sur l'échelle des arrondissements en remplaçant la valeur de "COMMUNE" par celle de "ARM" lorsque la ville en question est Paris, Marseille ou Lyon.

In [ ]:
# Création d'une table croisée dynamique pour compter les déplacements
# Il faut que la matrice soit carrée pour qu'on ait bien tous les couples possibles de communes
flux_tot = pd.crosstab(df_mobpro_brut['COMMUNE'], df_mobpro_brut['DCLT'], dropna=False)
all_values = pd.Index(sorted(set(df_mobpro_brut['COMMUNE']).union(df_mobpro_brut['DCLT'])))
flux_tot_carre = flux_tot.reindex(index=all_values, columns=all_values, fill_value=0)
flux_tot=flux_tot_carre

# Conversion en DataFrame pour plus de clarté
flux_tot = pd.DataFrame(flux_tot)

# Pour voir les dimensions de la matrice
print("Dimensions de la matrice :", flux_tot.shape)

In [ ]:
# Filtrer les données pour chaque groupe de transport
df_trans_45 = df_mobpro_brut[df_mobpro_brut['TRANS'].isin([4, 5])]
df_trans_6 = df_mobpro_brut[df_mobpro_brut['TRANS'] == 6]
df_trans_123 = df_mobpro_brut[df_mobpro_brut['TRANS'].isin([1, 2, 3])]

In [ ]:
import pandas as pd

communes_unique = df_trans_45['COMMUNE'].nunique()
dclt_unique = df_trans_45['DCLT'].nunique()
flux_rouge = pd.crosstab(df_trans_45['COMMUNE'], df_trans_45['DCLT'], dropna=False)
all_values = pd.Index(sorted(set(df_trans_45['COMMUNE']).union(df_trans_45['DCLT'])))
flux_rouge_carre = flux_rouge.reindex(index=all_values, columns=all_values, fill_value=0)
print(f"Dimensions après réindexation : {flux_rouge_carre.shape}")
flux_rouge = flux_rouge_carre


In [ ]:
# Créer la matrice des flux rouges

flux_rouge = pd.crosstab(df_trans_45['COMMUNE'], df_trans_45['DCLT'], dropna=False)
all_values = pd.Index(sorted(set(df_trans_45['COMMUNE']).union(df_trans_45['DCLT'])))
flux_rouge_carre = flux_rouge.reindex(index=all_values, columns=all_values, fill_value=0)
flux_rouge=flux_rouge_carre
print(flux_rouge.shape)

In [ ]:
# Parenthèse : sélection des villes sur le trjet de la ligne de tram simulée dans la partie simulation
print(flux_rouge.index)
codes_insee = [
    "95680",  # Villiers-le-Bel
    "95580",  # Sarcelles
    "95539",  # Saint-Brice-sous-Forêt
    "95197",  # Deuil-la-Barre
    "95427",  # Montmagny
    "93079",  # Villetaneuse
    "93031",  # Épinay-sur-Seine
    "92036",  # Gennevilliers
    "92009",  # Bois-Colombes
    "92035",  # La Garenne-Colombes
    "92050",  # Nanterre
    "92063"   # Rueil-Malmaison
]

selection_rouge=flux_rouge.loc[codes_insee, codes_insee]
print(selection_rouge)
selection_rouge.to_csv("selection_rouge")
pollution=7*0.14*(selection_rouge.sum().sum()-0.7*np.diag(selection_rouge).sum())*4*0.60
# 7 car on va considérer qu'en moyenne les gens qui vont prendre cette ligne vont faire un tiers du parcours de la ligne, qui fait 21 km de long
# 0.14 pcq la voiture c'est 140g de CO2 au km
# on somme tout sauf 70% de la diagonale pcq le tram ne fait pas 2 arrêts dans la même ville donc pas très utile pour la plupart des gens de le prendre quand il svont taffer dans la meme ville
# 4 pcq les poids sont généralement à 4
# et on va considérer que 60% des gens vont préférer le tram à la voiture pour faire ce trajet quand même vachement pénible
print(pollution)

In [ ]:
# Créer la matrice des flux jaunes

flux_jaune = pd.crosstab(df_trans_6['COMMUNE'], df_trans_6['DCLT'], dropna=False)
all_values = pd.Index(sorted(set(df_trans_6['COMMUNE']).union(df_trans_6['DCLT'])))
flux_jaune_carre = flux_jaune.reindex(index=all_values, columns=all_values, fill_value=0)
flux_jaune=flux_jaune_carre
print(flux_jaune.shape)

In [ ]:
# Créer la matrice des flux verts

flux_vert = pd.crosstab(df_trans_123['COMMUNE'], df_trans_123['DCLT'], dropna=False)
all_values = pd.Index(sorted(set(df_trans_123['COMMUNE']).union(df_trans_123['DCLT'])))
flux_vert_carre = flux_vert.reindex(index=all_values, columns=all_values, fill_value=0)
flux_vert=flux_vert_carre
print(flux_vert.shape)

In [ ]:
# Convertir en DataFrame pour plus de clarté
flux_rouge = pd.DataFrame(flux_rouge)
flux_jaune = pd.DataFrame(flux_jaune)
flux_vert = pd.DataFrame(flux_vert)

# Afficher les dimensions des matrices
print("Dimensions de la matrice (TRANS = 4 ou 5) :", flux_rouge.shape)
print("Dimensions de la matrice (TRANS = 6) :", flux_jaune.shape)
print("Dimensions de la matrice (TRANS = 1, 2 ou 3) :", flux_vert.shape)


In [ ]:
flux('78220', '75108', flux_jaune)

In [ ]:
flux_rouge

In [ ]:
# Pour les départs : somme sur l'axe des colonnes (chaque ligne = ville de départ)
df_flux_jaune_depart = flux_jaune.sum(axis=1).reset_index()
df_flux_jaune_depart.columns = ['COMMUNE', 'flux_depart']

# Pour les arrivées : somme sur l'axe des lignes (chaque colonne = ville d'arrivée)
df_flux_jaune_destination = flux_jaune.sum(axis=0).reset_index()
df_flux_jaune_destination.columns = ['DCLT', 'flux_destination']

print("Départ :", df_flux_jaune_depart.head())
print("Destination :", df_flux_jaune_destination.head())

df_flux_vert_depart = flux_vert.sum(axis=1).reset_index()
df_flux_vert_depart.columns = ['COMMUNE', 'flux_depart']

# Pour les arrivées : somme sur l'axe des lignes (chaque colonne = ville d'arrivée)
df_flux_vert_destination = flux_vert.sum(axis=0).reset_index()
df_flux_vert_destination.columns = ['DCLT', 'flux_destination']

print("Départ :", df_flux_vert_depart.head())
print("Destination :", df_flux_vert_destination.head())

df_flux_rouge_depart = flux_rouge.sum(axis=1).reset_index()
df_flux_rouge_depart.columns = ['COMMUNE', 'flux_depart']

# Pour les arrivées : somme sur l'axe des lignes (chaque colonne = ville d'arrivée)
df_flux_rouge_destination = flux_rouge.sum(axis=0).reset_index()
df_flux_rouge_destination.columns = ['DCLT', 'flux_destination']

print("Départ :", df_flux_rouge_depart.head())
print("Destination :", df_flux_rouge_destination.head())


In [ ]:
# Création d'un tableau superficies qui servira au calcul des émissions par villes
# En effet, la plupart des déplacements se font au sein d'une même ville En ayant la superficie des villes,
# on pourra calculer la distance moyenne entre deux points aléatoires tirés selon la probabilité uniforme 
# dans un cercle d'aire égale à la superficie de la ville (simplification arbitraire). 

# Conversion de geometry en un GeoDataFrame
contours_comm = gpd.GeoDataFrame(contours_comm)

# Reprojection des géométries en EPSG:2154 (Lambert 93, utilisé pour la France, mètres)
contours_comm = contours_comm.to_crs(epsg=2154)

# Calculer la surface des géométries en mètres carrés et conversion en km²
contours_comm['surface_m2'] = contours_comm.geometry.area
contours_comm['surface_km2'] = contours_comm['surface_m2'] / 1e6

# Création du tableau de superficies
superficies = contours_comm[['INSEE_COM', 'NOM', 'surface_km2']]

# Fusionner les communes des colonnes 'COMMUNE' et 'DCLT' dans df_mobpro_brut
communes_residence_travail = pd.concat([df_mobpro_brut['COMMUNE'], df_mobpro_brut['DCLT']]).drop_duplicates()

# Convertir la Série en DataFrame et renommer la colonne
communes_residence_travail = communes_residence_travail.to_frame(name='COMMUNE')

# Effectuer une jointure avec superficies sur les communes fusionnées
superficies = pd.merge(communes_residence_travail, superficies, left_on='COMMUNE', right_on='INSEE_COM', how='left')

print(f"Nombre de lignes dans le tableau final : {superficies.shape[0]}")
print(superficies)
# On a bien 34946 lignes, soit le nombre de communes en France : rien n'a été oublié

In [ ]:
'''J'ai eu un crash de kernel sur ce programme je ne comprends pas  pk'''

# # Calcul des sommes des lignes et des colonnes
#somme_lignes = flux_rouge.sum(axis=1).values  # on prend les valeurs sous forme de tableau numpy
#somme_colonnes = flux_rouge.sum(axis=0).values  # idem pour les colonnes

# # Créer la matrice nombre_total_de_commute en utilisant la diffusion de numpy (broadcasting)
# # Remplir une matrice de la forme de flux_rouge avec la somme des lignes et des colonnes
# nombre_total_de_commute = (somme_lignes[:, None] + somme_colonnes[None, :]) - flux_rouge.values

# # Transformer en DataFrame pour une présentation claire
# nombre_total_de_commute_df = pd.DataFrame(nombre_total_de_commute, columns=flux_rouge.columns, index=flux_rouge.index)

# # Afficher la matrice résultante
# print(nombre_total_de_commute_df)


In [ ]:
df_flux_jaune_depart_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_jaune_depart,
        left_on='INSEE_COM',
        right_on="COMMUNE",
        how='left')
df_flux_jaune_depart_m["flux_depart"] = df_flux_jaune_depart_m["flux_depart"].fillna(0).astype(int)

df_flux_vert_depart_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_vert_depart,
        left_on='INSEE_COM',
        right_on="COMMUNE",
        how='left')
df_flux_vert_depart_m["flux_depart"] = df_flux_vert_depart_m["flux_depart"].fillna(0).astype(int)

df_flux_rouge_depart_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_rouge_depart,
        left_on='INSEE_COM',
        right_on="COMMUNE",
        how='left')
df_flux_rouge_depart_m["flux_depart"] = df_flux_rouge_depart_m["flux_depart"].fillna(0).astype(int)


df_flux_jaune_destination_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_jaune_destination,
        left_on='INSEE_COM',
        right_on="DCLT",
        how='left')
df_flux_jaune_destination_m["flux_destination"] = df_flux_jaune_destination_m["flux_destination"].fillna(0).astype(int)

df_flux_vert_destination_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_vert_destination,
        left_on='INSEE_COM',
        right_on="DCLT",
        how='left')
df_flux_vert_destination_m["flux_destination"] = df_flux_vert_destination_m["flux_destination"].fillna(0).astype(int)

df_flux_rouge_destination_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_rouge_destination,
        left_on='INSEE_COM',
        right_on="DCLT",
        how='left')
df_flux_rouge_destination_m["flux_destination"] = df_flux_rouge_destination_m["flux_destination"].fillna(0).astype(int)

In [ ]:
""" OBTENTION DE final_df RECENSANT POUR CHAQUE VILLE TOUS LES TYPES DE FLUX ET INFOS SOCIO"""
""" OBTENTION DE df_epci_contours INDIQUANT POUR CHAQUE EPCI QUELQUES CARACTERISTIQUES ET SES CONTOURS"""

# Merge des données socio et contours communes
df_socio = pd.merge(
    contours_comm,
    df_dossier_complet_brut[['CODGEO', 'P21_SAL15P', 'C21_MEN', 'P21_ACTOCC15P_TP']],
    left_on='INSEE_COM',
    right_on='CODGEO',
    how='left')
    
# Fusionner tous les flux dans un unique DataFrame
final_df = df_socio[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry', 'P21_SAL15P', 'C21_MEN', 'P21_ACTOCC15P_TP']].copy()

# Ajouter les colonnes de flux en les fusionnant avec le DataFrame principal
for color in ['jaune', 'vert', 'rouge']:
    df_depart = globals()[f'df_flux_{color}_depart']
    df_destination = globals()[f'df_flux_{color}_destination']
    
    df_depart_m = pd.merge(final_df, df_depart, left_on='INSEE_COM', right_on='COMMUNE', how='left')
    df_destination_m = pd.merge(final_df, df_destination, left_on='INSEE_COM', right_on='DCLT', how='left')
    
    final_df[f'flux_depart_{color}'] = df_depart_m['flux_depart'].fillna(0).astype(int)
    final_df[f'flux_destination_{color}'] = df_destination_m['flux_destination'].fillna(0).astype(int)

# Renommer les colonnes
final_df = final_df.rename(columns={
    'P21_SAL15P': 'Nb actifs occupés en 2021',
    'C21_MEN': 'Nb ménages en 2021',
    'P21_ACTOCC15P_TP': 'Nb actifs à temps partiel'})

# Conversion en GeoDataFrame si nécessaire
if not isinstance(final_df, gpd.GeoDataFrame):
    final_df = gpd.GeoDataFrame(final_df, geometry='geometry')

# Partie données
df_epci_data = final_df.groupby('SIREN_EPCI').agg({
    'NOM': list,
    'INSEE_COM': list,
    'INSEE_DEP': list,
    'POPULATION': 'sum',
    'Nb actifs à temps partiel': 'sum',
    'Nb ménages en 2021':'sum',
    'Nb actifs occupés en 2021':'sum'
}).reset_index()

# Partie géométrie
df_epci_geo = final_df.dissolve(by='SIREN_EPCI')

# Merge des deux
df_epci_contours = df_epci_data.merge(df_epci_geo[['geometry']], left_on='SIREN_EPCI', right_index=True)


In [ ]:
df_epci_contours.head()

In [ ]:
plot_flux_gradient(df_flux_rouge_depart_m, "rouge", "Figure - Flux (domicile-travail) de départ \n en voiture/moto par commune","flux_depart")

In [ ]:
plot_flux_gradient(df_flux_jaune_depart_m, "jaune", "Figure - Flux (domicile-travail) de départ \n en transport en commun par commune","flux_depart")

In [ ]:
plot_flux_gradient(df_flux_vert_depart_m, "vert", "Figure - Flux (domicile-travail) de départ \n à pied","flux_depart")

In [ ]:
plot_flux_gradient(df_flux_rouge_destination_m, "rouge", "Figure - Flux (domicile-travail) d'arrivée \n en voiture/moto","flux_destination")

In [ ]:
plot_flux_gradient(df_flux_jaune_destination_m, "jaune", "Figure - Flux (domicile-travail) d'arrivée \n en transport en commun","flux_destination")

In [ ]:
plot_flux_gradient(df_flux_vert_destination_m, "vert", "Figure - Flux (domicile-travail) d'arrivée \n à pied","flux_destination")

In [ ]:
plot_flux_gradient_zoom(df_flux_jaune_destination_m, "jaune", "Figure - Flux (domicile-travail) d'arrivée \n en transports en commun","flux_destination", "grenoble")

In [ ]:
def est_mono_departement(liste_departements):
    '''
    On vérifie si les EPCI regroupent forcément des villes qui appartiennent au même département (réponse = non pas systématiquement)
    '''
    return len(set(liste_departements)) == 1
    
    
# Appliquer la fonction à chaque ligne et créer une nouvelle colonne
df_epci_contours['MONO_DEP'] = df_epci_contours['INSEE_DEP'].apply(est_mono_departement)

In [ ]:
"""OBTENTION DE gdf_epci_metropole IGNORANT LES EPCI HORS FRANCE METROPOLITAINE"""

def est_metropole(liste_departements):
    """
    On ne souhaite afficher que la France Métropolitaine et exclure les DOM-TOM et la Corse.
    """
    # Filtrage : ignorer les départements dont le code dépasse 3 caractères
    liste_departements = [dep for dep in liste_departements if len(str(dep)) <= 3]

    for dep in liste_departements:
        # On vérifie si le code contient des lettres, comme '2A' ou '2B'
        if any(c.isalpha() for c in dep):
            return False  # Exclure la Corse ou d'autres DOM-TOM avec des lettres

        # Comparer avec 970 pour la métropole
        if int(dep) >= 970:
            return False  # Exclure les départements DOM-TOM
    
    return True  # Si toutes les conditions sont remplies, c'est en métropole
    
gdf_epci_metropole = df_epci_contours[df_epci_contours['INSEE_DEP'].apply(est_metropole)]


In [ ]:
""" AFFICHAGE CARTE EPCI"""

gdf_epci_metropole = gpd.GeoDataFrame(gdf_epci_metropole, geometry='geometry')

# Tracer les géométries
fig, ax = plt.subplots(figsize=(10, 10))  # Taille de la figure
gdf_epci_metropole.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=1)
plt.title("Contours des EPCI Métropolitains")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
""" OBTENTION DE df_epci_flux QUI RECENSE LES NOMBRES DE FLUX PAR EPCI"""

# Merge pour récupérer EPCI de départ
df_mobpro_brut = df_mobpro_brut.merge(
    contours_comm[['INSEE_COM', 'SIREN_EPCI']],
    left_on='COMMUNE',
    right_on='INSEE_COM',
    how='left'
).rename(columns={'SIREN_EPCI': 'EPCI_DEP'}).drop('INSEE_COM', axis=1)

# Merge pour récupérer EPCI d'arrivée
df_mobpro_brut = df_mobpro_brut.merge(
    contours_comm[['INSEE_COM', 'SIREN_EPCI']],
    left_on='DCLT',
    right_on='INSEE_COM',
    how='left'
).rename(columns={'SIREN_EPCI': 'EPCI_ARR'}).drop('INSEE_COM', axis=1)

df_mobpro_brut['is_intra'] = df_mobpro_brut['EPCI_DEP'] == df_mobpro_brut['EPCI_ARR']

# Calcul des flux intra
flux_intra = df_mobpro_brut[df_mobpro_brut['is_intra']].groupby('EPCI_DEP').size().reset_index(name='flux_intra_epci')

# Calcul des flux sortants
flux_sortant = df_mobpro_brut[~df_mobpro_brut['is_intra']].groupby('EPCI_DEP').size().reset_index(name='flux_sortant')

# Calcul des flux entrants
flux_entrant = df_mobpro_brut[~df_mobpro_brut['is_intra']].groupby('EPCI_ARR').size().reset_index(name='flux_entrant')

# Calcul des flux par type de transport (TRANS = 6 : TC, TRANS = 1,2,3 : Zero, TRANS = 4,5 : VC2RM)
flux_sortant_TC = df_mobpro_brut[(df_mobpro_brut['TRANS'] == 6) & (~df_mobpro_brut['is_intra'])].groupby('EPCI_DEP').size().reset_index(name='flux_sortant_TC')
flux_entrant_TC = df_mobpro_brut[(df_mobpro_brut['TRANS'] == 6) & (~df_mobpro_brut['is_intra'])].groupby('EPCI_ARR').size().reset_index(name='flux_entrant_TC')
flux_sortant_VC2RM = df_mobpro_brut[(df_mobpro_brut['TRANS'].isin([4, 5])) & (~df_mobpro_brut['is_intra'])].groupby('EPCI_DEP').size().reset_index(name='flux_sortant_VC2RM')
flux_entrant_VC2RM = df_mobpro_brut[(df_mobpro_brut['TRANS'].isin([4, 5])) & (~df_mobpro_brut['is_intra'])].groupby('EPCI_ARR').size().reset_index(name='flux_entrant_VC2RM')
flux_sortant_Zero = df_mobpro_brut[(df_mobpro_brut['TRANS'].isin([1, 2, 3])) & (~df_mobpro_brut['is_intra'])].groupby('EPCI_DEP').size().reset_index(name='flux_sortant_Zero')
flux_entrant_Zero = df_mobpro_brut[(df_mobpro_brut['TRANS'].isin([1, 2, 3])) & (~df_mobpro_brut['is_intra'])].groupby('EPCI_ARR').size().reset_index(name='flux_entrant_Zero')

# Création du DataFrame final avec SIREN_EPCI comme clé unique
# Commencer par créer un DataFrame avec tous les EPCI uniques
all_epci = pd.concat([
    flux_intra[['EPCI_DEP']],
    flux_sortant[['EPCI_DEP']],
    flux_entrant[['EPCI_ARR']].rename(columns={'EPCI_ARR': 'EPCI_DEP'}),
    flux_sortant_TC[['EPCI_DEP']],
    flux_entrant_TC[['EPCI_ARR']].rename(columns={'EPCI_ARR': 'EPCI_DEP'}),
    flux_sortant_VC2RM[['EPCI_DEP']],
    flux_entrant_VC2RM[['EPCI_ARR']].rename(columns={'EPCI_ARR': 'EPCI_DEP'}),
    flux_sortant_Zero[['EPCI_DEP']],
    flux_entrant_Zero[['EPCI_ARR']].rename(columns={'EPCI_ARR': 'EPCI_DEP'})
]).drop_duplicates().rename(columns={'EPCI_DEP': 'SIREN_EPCI'})

# Fusion des flux dans un seul DataFrame
df_epci_flux = all_epci.copy()

# Fusionner tous les flux en utilisant des suffixes explicites et en maintenant SIREN_EPCI comme clé
df_epci_flux = pd.merge(df_epci_flux, flux_intra.rename(columns={'EPCI_DEP': 'SIREN_EPCI'}), 
                        on='SIREN_EPCI', how='left')

df_epci_flux = pd.merge(df_epci_flux, flux_sortant.rename(columns={'EPCI_DEP': 'SIREN_EPCI'}), 
                        on='SIREN_EPCI', how='left')

df_epci_flux = pd.merge(df_epci_flux, flux_entrant.rename(columns={'EPCI_ARR': 'SIREN_EPCI'}), 
                        on='SIREN_EPCI', how='left')

df_epci_flux = pd.merge(df_epci_flux, flux_sortant_TC.rename(columns={'EPCI_DEP': 'SIREN_EPCI'}), 
                        on='SIREN_EPCI', how='left')

df_epci_flux = pd.merge(df_epci_flux, flux_entrant_TC.rename(columns={'EPCI_ARR': 'SIREN_EPCI'}), 
                        on='SIREN_EPCI', how='left')

df_epci_flux = pd.merge(df_epci_flux, flux_sortant_VC2RM.rename(columns={'EPCI_DEP': 'SIREN_EPCI'}), 
                        on='SIREN_EPCI', how='left')

df_epci_flux = pd.merge(df_epci_flux, flux_entrant_VC2RM.rename(columns={'EPCI_ARR': 'SIREN_EPCI'}), 
                        on='SIREN_EPCI', how='left')

df_epci_flux = pd.merge(df_epci_flux, flux_sortant_Zero.rename(columns={'EPCI_DEP': 'SIREN_EPCI'}), 
                        on='SIREN_EPCI', how='left')

df_epci_flux = pd.merge(df_epci_flux, flux_entrant_Zero.rename(columns={'EPCI_ARR': 'SIREN_EPCI'}), 
                        on='SIREN_EPCI', how='left')

# Remplacer les NA par des 0
df_epci_flux = df_epci_flux.fillna(0)

# Convertir en int
for col in ['flux_intra_epci', 'flux_sortant', 'flux_entrant', 'flux_sortant_TC', 'flux_entrant_TC', 
            'flux_sortant_VC2RM', 'flux_entrant_VC2RM', 'flux_sortant_Zero', 'flux_entrant_Zero']:
    df_epci_flux[col] = df_epci_flux[col].astype(int)

# Résultat final
df_epci_flux.head()

In [ ]:
gdf_epci = gdf_epci_metropole.merge(df_epci_flux[['SIREN_EPCI', 'flux_intra_epci', 'flux_sortant', 'flux_entrant','flux_sortant_TC','flux_entrant_TC','flux_sortant_VC2RM','flux_entrant_VC2RM','flux_sortant_Zero','flux_entrant_Zero']], on='SIREN_EPCI', how='left')

In [ ]:
gdf_epci.sample()

In [ ]:
# Calcul du ratio (éviter la division par zéro)
gdf_epci['ratio'] = gdf_epci['flux_sortant'] / gdf_epci['flux_entrant'].replace(0, 1)

# Nettoyage de la colonne INSEE_DEP en supprimant les doublons dans les listes
gdf_epci['INSEE_DEP'] = gdf_epci['INSEE_DEP'].apply(lambda x: list(set(x)) if isinstance(x, list) else x)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# --- Exemple de données : chaque ligne est une ville, chaque colonne une variable ---
data = gdf_epci

# --- Sélection des variables à utiliser ---
selected_vars = ['ratio', 'Nb actifs occupés en 2021', 'flux_entrant_TC']  # On choisit les variables pertinentes
X = data[selected_vars].copy()  # On extrait les données

# --- Pondération des variables (facultatif) ---
weights = np.array([0.4, 0.3, 0.3])  # Importance de chaque variable
X_weighted = X * weights  # Multiplication élément par élément

# --- Normalisation des données ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_weighted)  # Mise à l'échelle pour équilibrer les unités

# --- Application du K-Means ---
k = 10  # Nombre de clusters (à adapter selon l'analyse)
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
data['cluster'] = kmeans.fit_predict(X_scaled)  # Ajout des clusters au DataFrame

# --- Affichage des résultats ---
print(data[['SIREN_EPCI', 'cluster']])

In [ ]:
import matplotlib.colors as mcolors

# Définir une palette de couleurs adaptée au nombre de clusters
num_clusters = data['cluster'].nunique()
colors = plt.cm.get_cmap('tab20', num_clusters)  # 'tab20', 'viridis', etc.

# Associer une couleur à chaque cluster
data['color'] = data['cluster'].apply(lambda x: colors(x))

# Tracer la carte
fig, ax = plt.subplots(figsize=(10, 10))
data.plot(column='cluster', cmap=colors, legend=True, edgecolor="black", ax=ax)

# Personnalisation
ax.set_title("Clustering des EPCI", fontsize=15)
ax.axis("off")  # Supprimer les axes

# Afficher la carte
plt.show()


In [ ]:
selected_vars = ['ratio', 'Nb actifs occupés en 2021', 'flux_entrant_TC']  
X = data[selected_vars].copy()

# --- Pondération des variables ---
weights = np.array([0.4, 0.3, 0.3])  # Importance de chaque variable
X_weighted = X * weights  # Multiplication élément par élément

# --- Normalisation des données ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- Test de plusieurs valeurs de k ---
inerties = []
k_values = range(1, 20)  # Tester de 1 à 20 clusters

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inerties.append(kmeans.inertia_)  # Stocker l'inertie

# --- Tracer la courbe du coude ---
plt.figure(figsize=(8, 5))
plt.plot(k_values, inerties, marker='o', linestyle='-')
plt.xlabel("Nombre de clusters (k)")
plt.ylabel("Inertie intra-cluster")
plt.title("Méthode du coude pour choisir k")
plt.xticks(k_values)
plt.grid()
plt.show()

In [ ]:
# --- Exemple de données : chaque ligne est une ville, chaque colonne une variable ---
data = gdf_epci

# --- Sélection des variables à utiliser ---
selected_vars = ['ratio', 'Nb actifs occupés en 2021', 'flux_entrant_TC']  # On choisit les variables pertinentes
X = data[selected_vars].copy()  # On extrait les données

# --- Pondération des variables (facultatif) ---
weights = np.array([0.4, 0.3, 0.3])  # Importance de chaque variable
X_weighted = X * weights  # Multiplication élément par élément

# --- Normalisation des données ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_weighted)  # Mise à l'échelle pour équilibrer les unités

# --- Application du K-Means ---
k = 5  # Nombre de clusters (à adapter selon l'analyse)
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
data['cluster'] = kmeans.fit_predict(X_scaled)  # Ajout des clusters au DataFrame

# Définir une palette de couleurs adaptée au nombre de clusters
num_clusters = data['cluster'].nunique()
colors = plt.cm.get_cmap('tab20', num_clusters)  # 'tab20', 'viridis', etc.

# Associer une couleur à chaque cluster
data['color'] = data['cluster'].apply(lambda x: colors(x))

# Tracer la carte
fig, ax = plt.subplots(figsize=(10, 10))
data.plot(column='cluster', cmap=colors, legend=True, edgecolor="black", ax=ax)

# Personnalisation
ax.set_title("Clustering des EPCI", fontsize=15)
ax.axis("off")  # Supprimer les axes

# Afficher la carte
plt.show()


In [ ]:
# Moyennes par cluster
cluster_summary_mean = data.groupby('cluster')[selected_vars].mean().reset_index()

print(cluster_summary_mean)


In [ ]:
recap = data.groupby('cluster').agg({
    'ratio': ['mean', 'std'],
    'Nb actifs occupés en 2021': ['mean', 'std'],
    'flux_entrant_TC': ['mean', 'std']
}).reset_index()

# Nommage des colonnes
recap.columns = ['cluster',
                 'ratio_mean', 'ratio_std',
                 'actifs_mean', 'actifs_std',
                 'flux_TC_mean', 'flux_TC_std']

In [ ]:
recap

In [ ]:
gdf_epci[gdf_epci["cluster"]==3]

In [ ]:
labels = {
    0: "Zones urbaines moyennes / Périphéries actives",
    1: "Petites villes / Zones rurales résidentielles",
    2: "Territoires résidentiels / Zones dortoirs éloignées",
    3: "Métropole / Pôle économique majeur",
    4: "Périphéries résidentielles / Zones semi-rurales en transition"
}

gdf_epci['cluster_label'] = gdf_epci['cluster'].map(labels)
